In [1]:
%matplotlib notebook
import cvxpy as cp
import torch
import numpy as np
from cvxpylayers.torch import CvxpyLayer
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import svm
from sklearn.metrics import zero_one_loss, confusion_matrix
from scipy.io import arff
import pandas as pd
import time
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.utils import shuffle
import matplotlib.patches as mpatches
import json
import random
import math
import os, psutil
from datetime import datetime

torch.set_default_dtype(torch.float64)
torch.manual_seed(0)
np.random.seed(0)

XDIM = 11
COST = 1/XDIM
TRAIN_SLOPE = 2
EVAL_SLOPE = 5
X_LOWER_BOUND = -10
X_UPPER_BOUND = 10

# Utils

In [2]:
def split_data(X, Y, percentage):
    num_val = int(len(X)*percentage)
    return X[num_val:], Y[num_val:], X[:num_val], Y[:num_val]

def shuffle(X, Y):
    data = torch.cat((X, Y), 1)
    data = data[torch.randperm(data.size()[0])]
    X = data[:, :2]
    Y = data[:, 2]
    return X, Y

def conf_mat(Y1, Y2):
    num_of_samples = len(Y1)
    mat = confusion_matrix(Y1, Y2, labels=[-1, 1])*100/num_of_samples
    acc = np.trace(mat)
    return mat, acc

def calc_accuracy(Y, Ypred):
    num = len(Y)
    temp = Y - Ypred
    acc = len(temp[temp == 0])*1./num
    return acc

# Dataset

In [3]:
def load_credit_default_data():
    torch.manual_seed(0)
    np.random.seed(0)
    url = 'https://raw.githubusercontent.com/ustunb/actionable-recourse/master/examples/paper/data/credit_processed.csv'
    df = pd.read_csv(url)
    df["NoDefaultNextMonth"].replace({0: -1}, inplace=True)
    df = df.sample(frac=1).reset_index(drop=True)

    df = df.drop(['Married', 'Single', 'Age_lt_25', 'Age_in_25_to_40', 'Age_in_40_to_59', 'Age_geq_60'], axis = 1)

    scaler = StandardScaler()
    df.loc[:, df.columns != "NoDefaultNextMonth"] = scaler.fit_transform(df.drop("NoDefaultNextMonth", axis=1))

    fraud_df = df.loc[df["NoDefaultNextMonth"] == -1]
    non_fraud_df = df.loc[df["NoDefaultNextMonth"] == 1][:6636]

    normal_distributed_df = pd.concat([fraud_df, non_fraud_df])

    # Shuffle dataframe rows
    df = normal_distributed_df.sample(frac=1).reset_index(drop=True)

    Y, X = df.iloc[:, 0].values, df.iloc[:, 1:].values
    return torch.from_numpy(X), torch.from_numpy(Y)

# CCP classes

In [4]:
class CCP:
    def __init__(self, x_dim, funcs):
        self.f_derivative = funcs["f_derivative"]
        self.g = funcs["g"]
        self.c = funcs["c"]
        
        self.x = cp.Variable(x_dim)
        self.xt = cp.Parameter(x_dim)
        self.r = cp.Parameter(x_dim)
        self.w = cp.Parameter(x_dim)
        self.b = cp.Parameter(1)
        self.slope = cp.Parameter(1)

        target = self.x@self.f_derivative(self.xt, self.w, self.b, self.slope)-self.g(self.x, self.w, self.b, self.slope)-self.c(self.x, self.r)
        constraints = [self.x >= X_LOWER_BOUND,
                       self.x <= X_UPPER_BOUND]
        self.prob = cp.Problem(cp.Maximize(target), constraints)
        
    def ccp(self, r):
        """
        numpy to numpy
        """
        self.xt.value = r
        self.r.value = r
        result = self.prob.solve()
        diff = np.linalg.norm(self.xt.value - self.x.value)
        cnt = 0
        while diff > 0.0001 and cnt < 10:
            cnt += 1
            self.xt.value = self.x.value
            result = self.prob.solve()
            diff = np.linalg.norm(self.x.value - self.xt.value)
        return self.x.value
    
    def optimize_X(self, X, w, b, slope):
        """
        tensor to tensor
        """
        w = w.detach().numpy()
        b = b.detach().numpy()
        slope = np.full(1, slope)
        X = X.numpy()
        
        self.w.value = w
        self.b.value = b
        self.slope.value = slope
        
        return torch.stack([torch.from_numpy(self.ccp(x)) for x in X])

In [5]:
class DELTA():
    
    def __init__(self, x_dim, funcs):
        self.g = funcs["g"]
        self.c = funcs["c"]
        
        self.x = cp.Variable(x_dim)
        self.r = cp.Parameter(x_dim, value = np.random.randn(x_dim))
        self.w = cp.Parameter(x_dim, value = np.random.randn(x_dim))
        self.b = cp.Parameter(1, value = np.random.randn(1))
        self.f_der = cp.Parameter(x_dim, value = np.random.randn(x_dim))

        target = self.x@self.f_der-self.g(self.x, self.w, self.b, TRAIN_SLOPE)-self.c(self.x, self.r)
        constraints = [self.x >= X_LOWER_BOUND,
                       self.x <= X_UPPER_BOUND]
        objective = cp.Maximize(target)
        problem = cp.Problem(objective, constraints)
        self.layer = CvxpyLayer(problem, parameters=[self.r, self.w, self.b, self.f_der],
                                variables=[self.x])
        
        
    def optimize_X(self, X, w, b, F_DER):
        return self.layer(X, w, b, F_DER)[0]

In [6]:
class BURDEN():
    
    def __init__(self, x_dim, funcs):
        self.c = funcs["c"]
        self.score = funcs["score"]

        self.x = cp.Variable(x_dim)
        self.r = cp.Parameter(x_dim, value = np.random.randn(x_dim))
        self.w = cp.Parameter(x_dim, value = np.random.randn(x_dim))
        self.b = cp.Parameter(1, value = np.random.randn(1))

        target = self.c(self.x, self.r)
        constraints = [self.score(self.x, self.w, self.b) >= 0,
                       self.x >= X_LOWER_BOUND,
                       self.x <= X_UPPER_BOUND]

        objective = cp.Minimize(target)
        problem = cp.Problem(objective, constraints)
        self.layer = CvxpyLayer(problem, parameters=[self.r, self.w, self.b], variables=[self.x])
        
        
    def calc_burden(self, X, Y, w, b):
        """
        tensor to tensor
        """
        Xpos = X[Y==1]
        if len(Xpos) == 0:
            return 0
        Xmin = self.layer(Xpos, w, b)[0]
        return torch.mean(torch.sum((Xpos-Xmin)**2, dim=1))

# Gain & Cost functions

In [7]:
def score(x, w, b):
    return x@w + b

def f(x, w, b, slope):
    return 0.5*cp.norm(cp.hstack([1, (slope*score(x, w, b) + 1)]), 2)

def g(x, w, b, slope):
    return 0.5*cp.norm(cp.hstack([1, (slope*score(x, w, b) - 1)]), 2)

def c(x, r):
    return COST*cp.sum_squares(x-r)

def f_derivative(x, w, b, slope):
    return 0.5*cp.multiply(slope*((slope*score(x, w, b) + 1)/cp.sqrt((slope*score(x, w, b) + 1)**2 + 1)), w)

funcs = {"f": f, "g": g, "f_derivative": f_derivative, "c": c, "score": score}

# Data generation

In [8]:
X, Y = load_credit_default_data()
X, Y = X[:3000], Y[:3000]

assert(len(X[0]) == XDIM)
X, Y, Xval, Yval = split_data(X, Y, 0.5)
Xval, Yval, Xtest, Ytest = split_data(Xval, Yval, 0.5)

print("percent of positive samples: {}%".format(100 * len(Y[Y == 1]) / len(Y)))

percent of positive samples: 49.266666666666666%


# Model

In [9]:
class MyStrategicModel(torch.nn.Module):
    def __init__(self, x_dim, funcs, train_slope, eval_slope, strategic=False, lamb=0):
        torch.manual_seed(0)
        np.random.seed(0)
        super(MyStrategicModel, self).__init__()
        self.x_dim = x_dim
        self.train_slope, self.eval_slope = train_slope, eval_slope
        self.w = torch.nn.parameter.Parameter(math.sqrt(1/x_dim)*(1-2*torch.rand(x_dim, dtype=torch.float64, requires_grad=True)))
        self.b = torch.nn.parameter.Parameter(torch.rand(1, dtype=torch.float64, requires_grad=True))
        self.strategic = strategic
        self.lamb = lamb
        self.ccp = CCP(x_dim, funcs)
        self.delta = DELTA(x_dim, funcs)
        self.burden = BURDEN(x_dim, funcs)

    def forward(self, X, evaluation=False):
        slope = self.eval_slope if evaluation else self.train_slope
        
        if self.strategic:
            XT = self.ccp.optimize_X(X, self.w, self.b, slope)
            F_DER = self.get_f_ders(XT, slope)
            X_opt = self.delta.optimize_X(X, self.w, self.b, F_DER) # Xopt should equal to XT but we do it again for the gradients

            output = self.score(X_opt)
        else:
            output = self.score(X)        
        
        return output
    
    def optimize_X(self, X, evaluation=False):
        slope = self.eval_slope if evaluation else self.train_slope
        return self.ccp.optimize_X(X, self.w, self.b, slope)
    
    def score(self, x):
        return x@self.w + self.b
    
    def get_f_ders(self, XT, slope):
        return torch.stack([0.5*slope*((slope*self.score(xt) + 1)/torch.sqrt((slope*self.score(xt) + 1)**2 + 1))*self.w for xt in XT])

    def evaluate(self, X, Y):
        Y_pred = torch.sign(self.forward(X, evaluation=True))
        num = len(Y)
        temp = Y - Y_pred
        acc = len(temp[temp == 0])*1./num        
        return acc
    
    def calc_burden(self, X, Y):
        return self.burden.calc_burden(X, Y, self.w, self.b)
    
    def loss(self, X, Y, Y_pred):
        if self.lamb > 0:
            return torch.mean(torch.clamp(1 - Y_pred * Y, min=0)) + self.lamb*self.calc_burden(X, Y)
        else:
            return torch.mean(torch.clamp(1 - Y_pred * Y, min=0))
        
    
    def save_model(self, X, Y, Xval, Yval, Xtest, Ytest, train_errors, val_errors, train_losses, val_losses, val_burdens, info, path, comment=None):
        if comment is not None:
            path += "_____" + comment
            
        filename = path + "/model.pt"
        if not os.path.exists(os.path.dirname(filename)):
            os.makedirs(os.path.dirname(filename))
        torch.save(self.state_dict(), filename)
        
        pd.DataFrame(X.numpy()).to_csv(path + '/X.csv')
        pd.DataFrame(Y.numpy()).to_csv(path + '/Y.csv')
        pd.DataFrame(Xval.numpy()).to_csv(path + '/Xval.csv')
        pd.DataFrame(Yval.numpy()).to_csv(path + '/Yval.csv')
        pd.DataFrame(Xval.numpy()).to_csv(path + '/Xtest.csv')
        pd.DataFrame(Yval.numpy()).to_csv(path + '/Ytest.csv')
        
        pd.DataFrame(np.array(train_errors)).to_csv(path + '/train_errors.csv')
        pd.DataFrame(np.array(val_errors)).to_csv(path + '/val_errors.csv')
        pd.DataFrame(np.array(train_losses)).to_csv(path + '/train_losses.csv')
        pd.DataFrame(np.array(val_losses)).to_csv(path + '/val_losses.csv')
        pd.DataFrame(np.array(val_burdens)).to_csv(path + '/val_burdens.csv')
        
        with open(path + "/info.txt", "w") as f:
            f.write(info)
    
    def load_model(self, filename):
        self.load_state_dict(torch.load(filename))
        self.eval()
    
    def fit(self, X, Y, Xval, Yval, Xtest, Ytest, opt, opt_kwargs={"lr":1e-3}, batch_size=128, epochs=100, verbose=False, callback=None, calc_train_errors=False, comment=None):
        train_dset = TensorDataset(X, Y)
        train_loader = DataLoader(train_dset, batch_size=batch_size, shuffle=True)
        opt = opt(self.parameters(), **opt_kwargs)

        train_losses = []
        val_losses = []
        train_errors = []
        val_errors = []
        val_burdens = []
        
        best_val_error = 1
        consecutive_no_improvement = 0
        now = datetime.now()
        path = "./models/burden/" + now.strftime("%d-%m-%Y_%H-%M-%S")

        total_time = time.time()
        for epoch in range(epochs):
            t1 = time.time()
            batch = 1
            train_losses.append([])
            train_errors.append([])
            for Xbatch, Ybatch in train_loader:
                opt.zero_grad()
                Ybatch_pred = self.forward(Xbatch)
                l = self.loss(Xbatch, Ybatch, Ybatch_pred)
                l.backward()
                opt.step()
                train_losses[-1].append(l.item())
                if calc_train_errors:
                    with torch.no_grad():
                        e = self.evaluate(Xbatch, Ybatch)
                        train_errors[-1].append(1-e)
                if verbose:
                    print("batch %03d / %03d | loss: %3.5f" %
                          (batch, len(train_loader), np.mean(train_losses[-1])))
                batch += 1
                if callback is not None:
                    callback()

            with torch.no_grad():
                Yval_pred = self.forward(Xval, evaluation=True)
                val_loss = self.loss(Xval, Yval, Yval_pred).item()
                val_losses.append(val_loss)
                val_error = 1-self.evaluate(Xval, Yval)
                val_errors.append(val_error)
                val_burden = self.calc_burden(Xval, Yval).item()
                val_burdens.append(val_burden)
                if val_error < best_val_error:
                    consecutive_no_improvement = 0
                    best_val_error = val_error
                    if self.strategic:
                        info = "training time in seconds: {}\nepoch: {}\nbatch size: {}\ntrain slope: {}\neval slope: {}\nlearning rate: {}\nvalidation loss: {}\nvalidation error: {}\nburden: {}".format(
                        time.time()-total_time, epoch, batch_size, self.train_slope, self.eval_slope, opt_kwargs["lr"], val_loss, val_error, val_burden)
                        self.save_model(X, Y, Xval, Yval, Xtest, Ytest, train_errors, val_errors, train_losses, val_losses, val_burdens, info, path, comment)
                        print("model saved!")
                else:
                    consecutive_no_improvement += 1
                    if consecutive_no_improvement >= 5:
                        break
                
            t2 = time.time()
            if verbose:
                print("----- epoch %03d / %03d | time: %03d sec | loss: %3.5f | err: %3.5f" % (epoch + 1, epochs, t2-t1, val_losses[-1], val_errors[-1]))
        print("training time: {} seconds".format(time.time()-total_time)) 
        return train_errors, val_errors, train_losses, val_losses

# Train

In [10]:
EPOCHS = 10
BATCH_SIZE = 64

x_dim = XDIM

# non-strategic classification
print("---------- training non-strategically----------")
non_strategic_model = MyStrategicModel(x_dim, funcs, TRAIN_SLOPE, EVAL_SLOPE, strategic=False)

fit_res_non_strategic = non_strategic_model.fit(X, Y, Xval, Yval, Xtest, Ytest,
                                opt=torch.optim.Adam, opt_kwargs={"lr": 5*(1e-2)},
                                batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=True, calc_train_errors=False)

lambda_range = torch.logspace(start=-2, end=0.1, steps=30)
print(lambda_range)
for lamb in lambda_range:

    # strategic classification
    print("---------- training strategically----------")
    print("lambda: ", lamb.item())
    strategic_model = MyStrategicModel(x_dim, funcs, TRAIN_SLOPE, EVAL_SLOPE, strategic=True, lamb=lamb)

    fit_res_strategic = strategic_model.fit(X, Y, Xval, Yval, Xtest, Ytest,
                                    opt=torch.optim.Adam, opt_kwargs={"lr": 5*(1e-2)},
                                    batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=True, calc_train_errors=False,
                                    comment="burden_" + str(lamb.item()))

    

---------- training non-strategically----------
batch 001 / 024 | loss: 1.02473
batch 002 / 024 | loss: 0.98493
batch 003 / 024 | loss: 0.94925
batch 004 / 024 | loss: 0.91794
batch 005 / 024 | loss: 0.87623
batch 006 / 024 | loss: 0.85815
batch 007 / 024 | loss: 0.83093
batch 008 / 024 | loss: 0.79413
batch 009 / 024 | loss: 0.79697
batch 010 / 024 | loss: 0.78610
batch 011 / 024 | loss: 0.77937
batch 012 / 024 | loss: 0.77649
batch 013 / 024 | loss: 0.78361
batch 014 / 024 | loss: 0.76675
batch 015 / 024 | loss: 0.75696
batch 016 / 024 | loss: 0.75645
batch 017 / 024 | loss: 0.74582
batch 018 / 024 | loss: 0.75056
batch 019 / 024 | loss: 0.74364
batch 020 / 024 | loss: 0.74202
batch 021 / 024 | loss: 0.73184
batch 022 / 024 | loss: 0.72664
batch 023 / 024 | loss: 0.72057
batch 024 / 024 | loss: 0.71777


C:\Users\barw1\anaconda3\envs\ranked_sc\lib\site-packages\cvxpy\expressions\expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 1 times so far.

  warnings.warn(msg, UserWarning)
C:\Users\barw1\anaconda3\envs\ranked_sc\lib\site-packages\cvxpy\expressions\expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 2 times so far.

  warnings.warn(msg, Use

----- epoch 001 / 010 | time: 001 sec | loss: 0.67070 | err: 0.30667
batch 001 / 024 | loss: 0.63756
batch 002 / 024 | loss: 0.74415
batch 003 / 024 | loss: 0.65346
batch 004 / 024 | loss: 0.64884
batch 005 / 024 | loss: 0.66767
batch 006 / 024 | loss: 0.69588
batch 007 / 024 | loss: 0.68155
batch 008 / 024 | loss: 0.68499
batch 009 / 024 | loss: 0.65439
batch 010 / 024 | loss: 0.64327
batch 011 / 024 | loss: 0.63350
batch 012 / 024 | loss: 0.62305
batch 013 / 024 | loss: 0.62653
batch 014 / 024 | loss: 0.62231
batch 015 / 024 | loss: 0.61317
batch 016 / 024 | loss: 0.60766
batch 017 / 024 | loss: 0.61433
batch 018 / 024 | loss: 0.61720
batch 019 / 024 | loss: 0.61618
batch 020 / 024 | loss: 0.61311
batch 021 / 024 | loss: 0.61944
batch 022 / 024 | loss: 0.62171
batch 023 / 024 | loss: 0.61818
batch 024 / 024 | loss: 0.62587
----- epoch 002 / 010 | time: 001 sec | loss: 0.64266 | err: 0.30667
batch 001 / 024 | loss: 0.62060
batch 002 / 024 | loss: 0.71489
batch 003 / 024 | loss: 0.7216

C:\Users\barw1\anaconda3\envs\ranked_sc\lib\site-packages\cvxpy\expressions\expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 4 times so far.

  warnings.warn(msg, UserWarning)
C:\Users\barw1\anaconda3\envs\ranked_sc\lib\site-packages\cvxpy\expressions\expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 5 times so far.

  warnings.warn(msg, Use

batch 001 / 024 | loss: 0.78666
batch 002 / 024 | loss: 0.88988
batch 003 / 024 | loss: 0.88344
batch 004 / 024 | loss: 0.89186
batch 005 / 024 | loss: 0.92062
batch 006 / 024 | loss: 0.92066
batch 007 / 024 | loss: 0.91981
batch 008 / 024 | loss: 0.90910
batch 009 / 024 | loss: 0.90979
batch 010 / 024 | loss: 0.90177
batch 011 / 024 | loss: 0.87971
batch 012 / 024 | loss: 0.87538
batch 013 / 024 | loss: 0.86987
batch 014 / 024 | loss: 0.86459
batch 015 / 024 | loss: 0.85140
batch 016 / 024 | loss: 0.84926
batch 017 / 024 | loss: 0.84182
batch 018 / 024 | loss: 0.84685


...

batch 009 / 024 | loss: 0.85466
batch 010 / 024 | loss: 0.87049
batch 011 / 024 | loss: 0.87962
batch 012 / 024 | loss: 0.89086
batch 013 / 024 | loss: 0.87874
batch 014 / 024 | loss: 0.88889
batch 015 / 024 | loss: 0.89418
batch 016 / 024 | loss: 0.89943
batch 017 / 024 | loss: 0.94123
batch 018 / 024 | loss: 0.94277
batch 019 / 024 | loss: 0.96575
batch 020 / 024 | loss: 0.96844
batch 021 / 024 | loss: 0.97496
batch 022 / 024 | loss: 0.96607
batch 023 / 024 | loss: 0.96133
batch 024 / 024 | loss: 0.96142
----- epoch 004 / 010 | time: 097 sec | loss: 0.89909 | err: 0.47733
batch 001 / 024 | loss: 1.80261
batch 002 / 024 | loss: 1.33273
batch 003 / 024 | loss: 1.24258
batch 004 / 024 | loss: 1.09818
batch 005 / 024 | loss: 1.06953
batch 006 / 024 | loss: 1.02528
batch 007 / 024 | loss: 1.01121
batch 008 / 024 | loss: 0.99616
batch 009 / 024 | loss: 0.99816
batch 010 / 024 | loss: 0.98450
batch 011 / 024 | loss: 0.97562
batch 012 / 024 | loss: 0.97463
batch 013 / 024 | loss: 0.95964
bat

C:\Users\barw1\anaconda3\envs\ranked_sc\lib\site-packages\cvxpy\expressions\expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 91 times so far.

  warnings.warn(msg, UserWarning)
C:\Users\barw1\anaconda3\envs\ranked_sc\lib\site-packages\cvxpy\expressions\expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 92 times so far.

  warnings.warn(msg, U

batch 001 / 024 | loss: 1.36105
batch 002 / 024 | loss: 1.59098
batch 003 / 024 | loss: 1.37703
batch 004 / 024 | loss: 1.26490
batch 005 / 024 | loss: 1.23142
batch 006 / 024 | loss: 1.20806
batch 007 / 024 | loss: 1.19293
batch 008 / 024 | loss: 1.16538
batch 009 / 024 | loss: 1.15635
batch 010 / 024 | loss: 1.14034
batch 011 / 024 | loss: 1.11015
batch 012 / 024 | loss: 1.11114
batch 013 / 024 | loss: 1.09953
batch 014 / 024 | loss: 1.13614
batch 015 / 024 | loss: 1.13183
batch 016 / 024 | loss: 1.12968
batch 017 / 024 | loss: 1.13332
batch 018 / 024 | loss: 1.14301
batch 019 / 024 | loss: 1.14244
batch 020 / 024 | loss: 1.13100
batch 021 / 024 | loss: 1.12165
batch 022 / 024 | loss: 1.11760
batch 023 / 024 | loss: 1.11533
batch 024 / 024 | loss: 1.10171
model saved!
----- epoch 001 / 010 | time: 118 sec | loss: 1.26773 | err: 0.47600
batch 001 / 024 | loss: 0.89890
batch 002 / 024 | loss: 0.94570
batch 003 / 024 | loss: 0.96370
batch 004 / 024 | loss: 0.93457
batch 005 / 024 | loss

C:\Users\barw1\anaconda3\envs\ranked_sc\lib\site-packages\cvxpy\problems\problem.py:1388: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "


----- epoch 004 / 010 | time: 098 sec | loss: 0.92314 | err: 0.47600
batch 001 / 024 | loss: 1.94561
batch 002 / 024 | loss: 1.40967
batch 003 / 024 | loss: 1.30240
batch 004 / 024 | loss: 1.15077
batch 005 / 024 | loss: 1.12382
batch 006 / 024 | loss: 1.07377
batch 007 / 024 | loss: 1.05230
batch 008 / 024 | loss: 1.02097
batch 009 / 024 | loss: 1.01964
batch 010 / 024 | loss: 1.00173
batch 011 / 024 | loss: 0.98970
batch 012 / 024 | loss: 0.99269
batch 013 / 024 | loss: 0.97551
batch 014 / 024 | loss: 0.96380
batch 015 / 024 | loss: 0.99651
batch 016 / 024 | loss: 0.99497
batch 017 / 024 | loss: 0.98170
batch 018 / 024 | loss: 0.97583
batch 019 / 024 | loss: 0.97333
batch 020 / 024 | loss: 0.97384
batch 021 / 024 | loss: 0.97486
batch 022 / 024 | loss: 0.97401
batch 023 / 024 | loss: 0.96800
batch 024 / 024 | loss: 0.96122
----- epoch 005 / 010 | time: 097 sec | loss: 0.96068 | err: 0.47600
batch 001 / 024 | loss: 0.90168
batch 002 / 024 | loss: 0.83545
batch 003 / 024 | loss: 0.8728